In [1]:
print("===> Import libraries")
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import numpy as np
from scipy import stats
from scipy.spatial import distance
from alibi_drift.AlibiDrift import AlibiDrift
from sklearn.datasets import make_blobs, make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.inspection import permutation_importance

import seaborn as sns
import matplotlib.pyplot as plt 
from matplotlib import style
%matplotlib inline
style.use("fivethirtyeight")

def kl_divergence(p, q):
    p = np.array(p)
    q = np.array(q)
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

In [2]:
print("\n===> Create train, test, val datasets")
X, y = make_classification(n_samples=400000, n_features=5, 
                           n_classes=2, flip_y=0.1, random_state=2)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=3/4, random_state=100)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=100) 
print(pd.value_counts(y))
print(np.mean(X), np.mean(y), X.shape, y.shape)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape)

<IPython.core.display.Javascript object>

0    200045
1    199955
dtype: int64
-0.0010136123088698708 0.4998875 (400000, 5) (400000,)
(80000, 5) (80000,) (300000, 5) (300000,) (20000, 5) (20000,)


In [3]:
%%time
print("\n===> Fit a RandomForestClassifier")
clf = RandomForestClassifier(random_state=2)
clf.fit(X_train, y_train)
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)
print("train accuracy: ", accuracy_score(y_train_pred, y_train))
print("test accuracy:  ", accuracy_score(y_test_pred, y_test))

train accuracy:  0.9999
test accuracy:   0.9201466666666667
CPU times: user 29.3 s, sys: 238 ms, total: 29.5 s
Wall time: 29.7 s


In [4]:
# %%time
# r = permutation_importance(clf, X_val, y_val, n_repeats=30, random_state=0)

# fnames = ["F"+str(i) for i in range(5)]
# for i in r.importances_mean.argsort()[::-1]:
#     #if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
#         print(f"{fnames[i]:<8}" 
#               f"{r.importances_mean[i]:.3f}"
#               f" +/- {r.importances_std[i]:.3f}")
        
# F4      0.272 +/- 0.002
# F0      0.240 +/- 0.002
# F3      0.124 +/- 0.002
# F2      0.083 +/- 0.001
# F1      -0.000 +/- 0.000